# DeepClassic

Reference
* [Asking RNNs+LTSMs: What Would Mozart Write?](http://www.wise.io/tech/asking-rnn-and-ltsm-what-would-mozart-write)

## music21 UserSetting
* http://web.mit.edu/music21/doc/tutorials/environment.html#environment
* [music21](https://gist.github.com/Vesnica/f8862277e4e3a27593f4ca300eedf07e)

### Install 

      sudo apt install musescore scipy timidity lilypond

In [ ]:
from music21 import *

In [ ]:
us = environment.UserSettings()
us.getSettingsPath()

In [ ]:
#us["musicxmlPath"] = "/usr/bin/gedit"
us["musicxmlPath"] = "/usr/bin/mscore"
us["midiPath"] = "/usr/bin/timidity"
us["showFormat"] = "lilypond"
us["writeFormat"] = "lilypond"
us["musescoreDirectPNGPath"] = "/usr/bin/mscore"

## Prepare Data

In [ ]:
!mkdir composer

In [ ]:
import glob
REP="@\n"
def trim_metadata(output_path, glob_path):
    comp_txt = open(output_path,"w")
    ll = glob.glob(glob_path)
    for song in ll:
        lines = open(song,"r").readlines()
        out = []
        found_first = False
        for l in lines:
            if l.startswith("="):
                ## new measure, replace the measure with the @ sign, not part of humdrum
                out.append(REP)
                found_first = True
                continue
            if not found_first:
                ## keep going until we find the end of the header and metadata
                continue
            if l.startswith("!"):
                ## ignore comments
                continue
            out.append(l)
        comp_txt.writelines(out)
    comp_txt.close()

### Get kern data from github database
*  [automata/ana-music: Automatic analysis of classical music for generative composition](https://github.com/automata/ana-music)

In [ ]:
!git clone https://github.com/automata/ana-music.git

In [ ]:
composers = ["mozart","beethoven","chopin","scarlatti","haydn"]
for composer in composers:
    output_path = "composer/" + composer + ".txt"
    glob_path = "ana-music/corpus/{composer}/*.krn".format(composer=composer)
    trim_metadata(output_path, glob_path)

In [ ]:
!ls composer/*.txt

### Get Data from KernScore
* [KernScores](http://kern.humdrum.org/)

In [ ]:
%mkdir kernscore
%mkdir kernscore/bach

In [ ]:
from urllib.request import urlopen
for i in range(1,15+1):
    filename = "inven{0:02d}.krn".format(i)
    file = urlopen("http://kern.humdrum.org/cgi-bin/ksdata?l=osu/classical/bach/inventions&file=%s&f=kern"%filename)
    with open("kernscore/bach/"+filename,'wb') as output:
        output.write(file.read())

In [ ]:
output_path = "composer/bach.txt"
glob_path = "kernscore/bach/*.krn"
trim_metadata(output_path, glob_path)

## Setup

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

In [2]:
filename = 'composer/bach.txt'
with open(filename, 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [3]:
text[:50]

'@\n4.r\t16dL\n.\t16e\n.\t16f\n.\t16g\n.\t16a\n.\t16b-J\n@\n4.r\t1'

In [4]:
encoded[:100]

array([21, 11, 16, 25,  8, 19, 36,  2, 18, 10, 11, 25, 19, 36,  2, 20, 11,
       25, 19, 36,  2, 34, 11, 25, 19, 36,  2,  4, 11, 25, 19, 36,  2, 32,
       11, 25, 19, 36,  2, 12,  5, 37, 11, 21, 11, 16, 25,  8, 19, 36,  2,
       22, 38, 10, 11, 25, 19, 36,  2, 12,  5, 11, 25, 19, 36,  2, 32, 11,
       25, 19, 36,  2,  4, 11, 25, 19, 36,  2, 34, 11, 25, 19, 36,  2, 20,
       37, 11, 21, 11, 36,  2, 17, 10, 19, 27, 34, 10, 11, 36,  2], dtype=int32)

In [5]:
vocab_size = len(vocab)
vocab_size

43

## Preprocess

In [6]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the batch size and number of batches we can make
    batch_size = n_seqs * n_steps
    n_batches = len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

In [7]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [8]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[21 11 16 25  8 19 36  2 18 10]
 [19 36  2  4  4 13 31 11 36  2]
 [36  2 22 22 38 11 25 19 36  2]
 [12 12 11 36  2 39 19 25 11 21]
 [ 2  4 11 36  2 23 19 36  2 34]
 [32 32 11 36  2 18 19 25 11 36]
 [24 15 17 38 19 36  2 34 38 37]
 [19 25 11 36  2 39 19 16 12  5]
 [18 37 11 21 11 36  2 23 10 19]
 [36  2  4 11 21 11 36  2 42 19]]

y
 [[11 16 25  8 19 36  2 18 10 11]
 [36  2  4  4 13 31 11 36  2  0]
 [ 2 22 22 38 11 25 19 36  2 32]
 [12 11 36  2 39 19 25 11 21 11]
 [ 4 11 36  2 23 19 36  2 34 38]
 [32 11 36  2 18 19 25 11 36  2]
 [15 17 38 19 36  2 34 38 37 11]
 [25 11 36  2 39 19 16 12  5 28]
 [37 11 21 11 36  2 23 10 19 27]
 [ 2  4 11 21 11 36  2 42 19 27]]


## Build Model

In [9]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

In [10]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    # Use a basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state


In [11]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        x: Input tensor
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # That is, the shape should be batch_size*num_steps rows by lstm_size columns
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

In [12]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per batch_size per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

In [13]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [14]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

##  Training

In [15]:
batch_size = 100        # Sequences per batch
num_steps = 100         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability

In [18]:
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/20...  Training Step: 1...  Training loss: 3.7573...  3.3294 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 3.6305...  3.2960 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 3.2176...  3.2646 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 3.2395...  3.2596 sec/batch
Epoch: 2/20...  Training Step: 5...  Training loss: 3.2303...  3.2477 sec/batch
Epoch: 2/20...  Training Step: 6...  Training loss: 3.0707...  3.2408 sec/batch
Epoch: 2/20...  Training Step: 7...  Training loss: 3.0182...  3.2367 sec/batch
Epoch: 2/20...  Training Step: 8...  Training loss: 3.0284...  3.2586 sec/batch
Epoch: 3/20...  Training Step: 9...  Training loss: 3.0550...  3.2530 sec/batch
Epoch: 3/20...  Training Step: 10...  Training loss: 2.9846...  3.2664 sec/batch
Epoch: 3/20...  Training Step: 11...  Training loss: 2.9887...  3.2661 sec/batch
Epoch: 3/20...  Training Step: 12...  Training loss: 2.9697...  3.2426 sec/batch
Epoch: 4/20...  Training Step: 13... 

In [19]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i80_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i80_l512.ckpt"

## Sampling

In [20]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [21]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [22]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i80_l512.ckpt'

In [23]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="@\n")
print(samp)

@
.1A6	1ddc#
16		
ec
	16dc	16
	16d
16A	16ed
1BB	16f

16B	..166-
16A		
cecc16c	.
16B		6

16		8e
@16G	.16e
116	#..166c#
611	f
.16e	16e
16G	.
16e-	16d
16B	.16f
1	dd16f
816	16
16	.
16A	16cc.
16	-#
8A		6c
8@	8
16A	..16c	#16

8B	16fc
16B	8dd16e-	16de
16G	16c-
16G	.16ec
16	e16d
.	16e-#8
16	16dc
16B	86
81A-	.
16A		16fc

6	16a
16F	.16g
16A	8

B	Ad
8		.
8c	8
.	16e
.	16ed
8A	16fd
816		
d
16AA	..16fd8
16	-#
.	16ge

	66c#.
16A		6c

16		.
116A.	8
16e	8.
8A	16g#..
16A	16d
	16ce
.16B	8c
16G	.
66	16c

16		.
16cB	8
16B		
c16c#	.
16G	..616d	16ee
16	-
16	#
16B	
16B	
6e16B	16d#
16		16d
.16	#16c#
	116c	.
16B--	16d-
116	#
16A	16d
	16c-.816d-	16e

16G		6fd
16F		16f
.16A	16c-
16F	16d
16A	16d
.16B	16e
8	16f

	66e
.	16ed
.16	16e
.	16ec
	66c
.16		
1dc
.16A.	16fc
16		16
e
.	16c#
16	1dd
16F	16d-16c
16A	16f-

6	16f-
.1AB	16e
.	16e#
8	16ee
.	16de
	61ec
.	16c#.
16	16d
16	16e
16B	.16c#
16G	16c-
16	-
16B	16f
16B	16f
	16d
816B	16d

	6e-16c	16
#16B	16c#	16e-
	16dd
16G	16d
16F	16fc
16AA	16d-.16c	16de
16A-	16c
16	-
16c#	1dd

In [36]:
f = open("results/m1a.krn","r").readlines()
r = []
bar = 1
for l in f:
    if l.startswith("@"):
        if bar == 1:
            r.append("=1-\t=1-\t=1-\n")
        else:
            r.append("={bar}\t={bar}\t={bar}\n".format(bar=bar))
        bar += 1
    else:
        r.append(l)
open("results/m1a-bar.krn","w").writelines(r)

In [37]:
from music21 import *
m1 = converter.parse("results/m1a-bar.krn")

ValueError: not enough values to unpack (expected 2, got 1)